In [1]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
default_model = os.getenv("OPENAI_DEFAULT_MODEL")
redis_host = os.getenv("REDIS_HOST")
redis_password = os.getenv("REDIS_PASSWORD")
redis_port = os.getenv("REDIS_PORT")
tavily_key = os.getenv("TAVILY_API_KEY")


In [2]:
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated, List, Literal
from dotenv import load_dotenv
import os
from langgraph.types import interrupt, Command
from operator import add

c:\Users\edukd\miniconda3\envs\myKDA2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm = ChatOpenAI(api_key=api_key, model_name=default_model)

In [4]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


In [6]:
from langchain_core.messages import AIMessage

def chatbot1(state: State):
   # 가장 최근 메시지 가져오기
    last_msg = state["messages"][-1]
    
    # 메시지 내용에 "도움"이라는 단어가 있는지 확인
    if "도움" in last_msg.content:
        human_input = interrupt({"query": "이 질문에 어떻게 답해야 할까요?"})
        response = AIMessage(content=human_input.get("data", "답변 없음"))
    else:        
        llm_response = llm.invoke([last_msg])  # LLM의 메시지 생성
        response = AIMessage(content=llm_response.content)
    
    return {"messages": [response]}


In [11]:
workflow = StateGraph(State)

workflow.add_node("chatbot1", chatbot1)

workflow.add_edge(START, "chatbot1") 
workflow.add_edge("chatbot1", END)    

# MemorySaver: 메모리에 그래프 상태를 저장하는 checkpointer
# checkpointer가 있어야 interrupt 후 재개가 가능함
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)


In [15]:
config = {"configurable":{"thread_id":"1"}}

In [19]:
state = {"messages":[HumanMessage(content="Langchain에 대해 설명해줘")]}
result = graph.invoke(state,config)
print(result)

LangChain은 주로 자연어 처리(NLP) 애플리케이션을 구축하기 위한 프레임워크로, 특히 대화형 AI 및 언어 모델을 기반으로 한 시스템을 개발하는 데 도움을 주기 위해 설계되었습니다. 이 프레임워크는 텍스트 처리, 데이터 관리, 다양한 NLP 작업을 위한 여러 모듈과 구성 요소를 제공합니다.

### 주요 특징 및 구성 요소:
1. **모듈화**: LangChain은 다양한 구성 요소로 모듈화되어 있어 개발자가 필요에 따라 쉽게 추가하거나 변경할 수 있습니다.
  
2. **프롬프트 관리**: 언어 모델에 전달할 프롬프트를 쉽게 생성하고 관리할 수 있는 도구를 제공합니다. 이를 통해 사용자는 모델의 출력 형태나 내용에 맞게 프롬프트를 조정할 수 있습니다.

3. **체인**: LangChain에서는 여러 작업을 연결하여 체인을 형성할 수 있습니다. 예를 들어, 데이터베이스에서 데이터를 가져오고, 이를 처리한 후, 언어 모델에 전달하는 과정이 하나의 체인으로 통합될 수 있습니다.

4. **텍스트 생성**: 텍스트 생성 모델과의 통합이 용이하여, 대화형 인터페이스나 콘텐츠 생성 등에 활용할 수 있습니다.

5. **유연한 백엔드**: 여러 개의 언어 모델 API와 통합 가능하여, OpenAI GPT, Hugging Face Transformers 등 다양한 모델을 사용할 수 있습니다.

6. **검증 및 평가**: 모델의 응답을 검증하거나 평가하는 도구를 포함하여, 생성된 콘텐츠의 품질을 체크할 수 있는 기능도 제공합니다.

### 적용 사례:
- **챗봇**: 자연어 대화를 이해하고 적절한 응답을 생성하는 챗봇 구축.
- **질문-답변 시스템**: 데이터베이스나 문서에서 정보를 가져와 질문에 대한 응답을 생성.
- **콘텐츠 생성**: 블로그 포스트, 기사, 마케팅 자료 등의 자동 생성.

LangChain은 대화형 AI 애플리케이션을 쉽게 구축하고 배포할 수 있도록 설계되어, 개발자들 사이에서 인기를 끌고 있습니다.
{'messages': [Human

In [20]:
state = graph.get_state(config)

print(f"실행 중단됨: {bool(state.next)}")
print(f"다음 노드{state.next}")

실행 중단됨: False
다음 노드()


In [14]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt, Command
from typing import TypedDict, Annotated, Literal
from operator import add
from langchain_core.messages import HumanMessage, AIMessage
# ============================================
# 1. State 정의
# ============================================
class State(TypedDict):
    """그래프에서 사용할 상태 정의
    messages: 대화 메시지 리스트
    Annotated[list, add]: 새 메시지가 기존 리스트에 추가됨
    """
    messages: Annotated[list, add]
# ============================================
# 2. 노드 함수 정의
# ============================================
def chatbot(state: State):
    # 가장 최근 메시지 가져오기
    last_msg = state["messages"][-1]
    # 메시지 내용에 "도움"이라는 단어가 있는지 확인
    if "도움" in last_msg.content:
        human_input = interrupt({"query": "이 질문에 어떻게 답해야 할까요?"})
        response = AIMessage(content=human_input.get("data", "답변 없음"))
    else:
        # ====== 수정 부분 시작 ======
        # 여기에 LLM 호출 코드 추가 (llm은 이미 선언되어 있다고 가정)
        llm_response = llm.invoke([last_msg])  # LLM의 메시지 생성
        response = AIMessage(content=llm_response.content)
    # 새 메시지를 상태에 추가 (Annotated[list, add]로 자동 병합됨)
    print(response.content)
    return {"messages": [response]}
# ============================================
# 3. 워크플로우 구성
# ============================================
# StateGraph 생성: State 타입을 사용하는 그래프
workflow = StateGraph(State)
# 노드 추가: "chatbot"이라는 이름으로 chatbot 함수 등록
workflow.add_node("chatbot", chatbot)
# 엣지 추가: START -> chatbot -> END 순서로 실행
workflow.add_edge(START, "chatbot")  # 시작점에서 chatbot으로
workflow.add_edge("chatbot", END)    # chatbot에서 종료점으로
# ============================================
# 4. 그래프 컴파일 (Checkpointer 필수!)
# ============================================
# MemorySaver: 메모리에 그래프 상태를 저장하는 checkpointer
# checkpointer가 있어야 interrupt 후 재개가 가능함
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)
# ============================================
# 5. 그래프 실행
# ============================================
# config: 그래프 실행 설정
# thread_id: 대화 세션을 구분하는 ID (같은 ID로 재개해야 함)
config = {"configurable": {"thread_id": "1"}}
print("=== 첫 실행 ===")
# invoke: 그래프를 동기적으로 실행
# HumanMessage: 사용자가 보낸 메시지
# "도움"이 포함되어 있어서 interrupt가 발생할 것임
#도움이 필요합니다 / LangChain에 대해 설명해줘
result = graph.invoke(
    {"messages": [HumanMessage(content="도움이 필요합니다 / LangChain에 대해 설명해줘")]},
    config
)
# ============================================
# 6. 실행 상태 확인
# ============================================
# get_state: 현재 그래프의 상태 조회
state = graph.get_state(config)
# state.next: 다음에 실행될 노드 리스트
# 비어있지 않으면 interrupt로 중단된 상태
print(f"실행 중단됨: {bool(state.next)}")
print(f"다음 노드: {state.next}")
# ============================================
# 7. Interrupt 재개 (인간 응답 제공)
# ============================================
# interrupt가 발생했는지 확인
if state.next:
    print("\n=== 인간 응답으로 재개 ===")
    # 인간이 제공하는 실제 응답
    human_response = "네, LangGraph는 훌륭한 선택입니다!"
    # Command(resume=...): 중단된 지점에서 실행 재개
    # resume 딕셔너리의 데이터가 interrupt()의 반환값이 됨
    result = graph.invoke(
        Command(resume={"data": human_response}),
        config  # 같은 thread_id 사용!
    )
    # 최종 결과 출력
    # messages[-1]: 가장 최근 메시지 (AI의 최종 응답)
    print(f"최종 응답: {result['messages'][-1].content}")

=== 첫 실행 ===
실행 중단됨: True
다음 노드: ('chatbot',)

=== 인간 응답으로 재개 ===
네, LangGraph는 훌륭한 선택입니다!
최종 응답: 네, LangGraph는 훌륭한 선택입니다!
